# Step 4-0. LLM 이미지 분석 (RAG 파이프라인)
텍스트로 이미지를 검색한 후, Claude LLM에게 이미지를 분석하도록 요청합니다.

## 배경 지식

### RAG (Retrieval-Augmented Generation)
- **RAG = 검색(Retrieval) + 생성(Generation)**
- 기존 LLM은 학습된 데이터만 기반으로 답변을 생성하지만, RAG는 **외부 지식을 검색해서 참고**한 뒤 더 정확한 답변을 생성
- 이 실습에서는: 사용자 질문 → OpenSearch 벡터 검색 → 검색된 이미지를 LLM에 전달 → 이미지 설명 생성

### LLM에 외부 데이터를 주입하는 3가지 방법
1. **Prompt Engineering**: 필요한 값을 프롬프트에 직접 입력
2. **RAG**: DB에서 유사한 데이터만 추출해서 프롬프트에 넣기 (프롬프트 길이 제한 대응)
3. **Fine Tuning**: 데이터가 너무 많은 경우 모델 자체를 학습 (비용이 비싸고 최소 10MB 이상 데이터 필요)

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [7]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

# 멀티모달 검색 설정 (Nova)
MULTIMODAL_INDEX_NAME = 'nova-image-test'
MULTIMODAL_EMBEDDING_MODEL_ID = 'amazon.nova-2-multimodal-embeddings-v1:0'
VECTOR_DIMENSION = 1024

# LLM 설정
# LLM_MODEL_ID = "us.anthropic.claude-opus-4-6-v1"
LLM_MODEL_ID = "us.anthropic.claude-sonnet-4-5-20250929-v1:0"

✅ config.json 로드 완료


## 2. OpenSearch 클라이언트 생성

In [8]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 함수 정의

In [9]:
import json
import base64
import math
from opensearchpy.exceptions import RequestError

session = boto3.Session(profile_name=PROFILE)
bedrock_client = session.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def normalize_vector(vec):
    """벡터를 L2 정규화합니다."""
    norm = math.sqrt(sum(x * x for x in vec))
    return [x / norm for x in vec] if norm > 0 else vec

def get_text_embedding(text, model_id):
    """Nova 멀티모달 모델을 사용하여 텍스트를 벡터로 변환합니다."""
    body = json.dumps({
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingPurpose": "GENERIC_RETRIEVAL",
            "embeddingDimension": VECTOR_DIMENSION,
            "text": {
                "truncationMode": "END",
                "value": text
            }
        }
    })
    response = bedrock_client.invoke_model(
        body=body, modelId=model_id, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    return normalize_vector(response_body["embeddings"][0]["embedding"])

def search_similar_image(index_name, query_vector):
    """OpenSearch에서 벡터 검색을 수행하여 가장 유사한 이미지 경로를 반환합니다."""
    search_query = {
        "size": 1,
        "query": {"knn": {"content_vector": {"vector": query_vector, "k": 1}}}
    }
    try:
        response = client.search(index=index_name, body=search_query)
        hits = response['hits']['hits']
        if not hits:
            return None
        return hits[0]['_source'].get('image_path')
    except RequestError as e:
        print(f"Error during OpenSearch search: {e.info}")
        return None

def image_to_base64(image_path):
    """이미지 파일을 Base64로 인코딩합니다."""
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        return None
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image {image_path}: {e}")
        return None

def get_image_description_from_llm(base64_image, user_question):
    """Claude Sonnet 모델에게 이미지를 보여주고 설명을 요청합니다."""
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 300,
        "temperature": 0.7,
        "top_p": 0.9,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {"type": "base64", "media_type": "image/jpeg", "data": base64_image},
                    },
                    {
                        "type": "text",
                        "text": f"이 이미지는 '{user_question}'이라는 질문으로 검색된 결과입니다. 이미지에 대해 친절하고 상세하게 설명해주세요."
                    }
                ],
            }
        ]
    })

    response = bedrock_client.invoke_model(
        body=body, modelId=LLM_MODEL_ID, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    return response_body['content'][0]['text']

Bedrock client created successfully.


In [10]:
# 인덱스에 데이터가 업로드되었는지 확인
if not client.indices.exists(index=MULTIMODAL_INDEX_NAME):
    print(f"❌ '{MULTIMODAL_INDEX_NAME}' 인덱스가 존재하지 않습니다.")
    print("   → Step 3-2b (Nova 이미지 임베딩 데이터 업로드) 노트북을 먼저 실행해주세요.")
else:
    _doc_count = client.count(index=MULTIMODAL_INDEX_NAME)['count']
    if _doc_count == 0:
        print(f"⏳ '{MULTIMODAL_INDEX_NAME}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
        print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
    else:
        print(f"✅ '{MULTIMODAL_INDEX_NAME}' 인덱스에 {_doc_count}개의 문서가 준비되어 있습니다.")

⏳ 'nova-image-test' 인덱스는 있지만 검색 가능한 문서가 0개입니다.
   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.


## 4. 실행: 이미지 검색 + LLM 분석

In [11]:
# Nova 모델은 한국어를 지원하므로 한국어로 검색합니다
# TODO: 검색할 질문을 바꿔보세요!
user_query = "꽃 위에 벌"

# 1. 텍스트-이미지 검색
print(f"\n1. '{user_query}'와(과) 유사한 이미지를 검색합니다...")
query_vector = get_text_embedding(user_query, MULTIMODAL_EMBEDDING_MODEL_ID)

found_image_path = search_similar_image(MULTIMODAL_INDEX_NAME, query_vector)
if not found_image_path:
    print("관련 이미지를 찾지 못했습니다.")
else:
    print(f"   - 이미지 찾음: {found_image_path}")

    # 2. 이미지 분석 및 설명 생성
    print("\n2. 찾은 이미지를 분석하여 설명을 생성합니다...")
    base64_image_data = image_to_base64(found_image_path)

    description = get_image_description_from_llm(base64_image_data, user_query)

    # 3. 최종 결과 출력
    print("\n--- 최종 결과 ---")
    print(f"검색된 이미지: {found_image_path}")
    print(f"\n이미지 설명:")
    print(description)


1. '꽃 위에 벌'와(과) 유사한 이미지를 검색합니다...
관련 이미지를 찾지 못했습니다.


## 실습 과제

### 과제: 다양한 LLM으로 테스트하고 결과와 응답 속도를 비교해보기
위 설정 셀에서 `LLM_MODEL_ID`를 변경하여 다시 실행해보세요.

| 모델 | Model ID | 특징 |
|------|----------|------|
| Claude Sonnet 4.5 | `us.anthropic.claude-sonnet-4-5-20250929-v1:0` | 균형 잡힌 성능 (기본) |
| Claude Opus 4.6 | `us.anthropic.claude-opus-4-6-v1` | 최고 성능, 느림 |

- `user_query`도 다양하게 바꿔보며 테스트해보세요